# Single Base Training on GPU1

# Quick Links <a name = "Top"></a>
<ol>
    <li><p><a href = #setup>Set Up</a></p></li>
    <li><p><a href = #Base>Base Training</a></p></li>
</ol>

# Imports

In [4]:
import os
import shutil

print('Current Conda Environment: {}'.format(os.environ['CONDA_DEFAULT_ENV']))

Current Conda Environment: tf_ks


In [5]:
#pip install talos

In [6]:
import talos as ta
from talos.model import lr_normalizer, early_stopper, hidden_layers

import tensorflow as tf
  
available_gpus = tf.config.experimental.list_physical_devices('GPU')
built_with_cuda = tf.test.is_built_with_cuda()

if not (not available_gpus) & built_with_cuda:
    print("The installed version of TensorFlow {} includes GPU support.\n".format(tf.__version__))
    print("Num GPUs Available: ", len(available_gpus), "\n")
else:
    print("The installed version of TensorFlow {} does not include GPU support.\n".format(tf.__version__))
    
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

from tensorflow.compat.v1.keras import callbacks, backend as K
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop, SGD, Adagrad
from tensorflow.keras.layers import ReLU, LeakyReLU

from datetime import datetime
import pandas as pd
import numpy as np

import time

from numpy.random import seed
seed(1)
tf.random.set_seed(1)

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
config.gpu_options.per_process_gpu_memory_fraction = 0.99
sess = tf.compat.v1.Session(config = config)

#with tf.compat.v1.Session(config = config) as sess: 
#    sess.run()

K.set_session(sess)

Using TensorFlow backend.


The installed version of TensorFlow 2.1.0 includes GPU support.

Num GPUs Available:  2 

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2590557961705015676
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9105744200
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1683431280603452536
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 9104897474
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16309785601070220303
physical_device_desc: "device: 1, name: GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5"
]


# Hilfsfunktionen

### Output Directory

* <i>SSD</i>, falls genug Speicher auf SSD im SymLink <i>fast_output</i> verfügbar ist
* <i>HDD</i>, falls möglicherweise zu wenig SSD-Speicher verfügbar ist $\rightarrow$ <i>output</i>

In [7]:
from enum import IntEnum

class OutputDirectory(IntEnum):
    HDD = 0
    SSD = 1
    
output_path = ['output', 'fast_output']

### Convert Label_Type into suitable label names.
$\Rightarrow$ Angular / Normalized $\rightarrow$ ['Elevation', 'Azimuth']

$\Rightarrow$ Stereographic $\rightarrow$ ['S_x', 'S_y']

In [8]:
def get_Label_Names(label_type):
    if label_type == 'Angular' or label_type == 'Normalized':
        return ['Elevation', 'Azimuth']
    elif label_type == 'Stereographic':
        return ['S_x', 'S_y']
    else:
        assert(True, 'LabelType Invalid')
        return None

### Benutzerdefinierte Kostenfunktion & Metrik

In [9]:
def circular_mse(y_true, y_pred):
    max_error = tf.constant(360, dtype = 'float32')
    return K.mean(K.square(K.minimum(K.abs(y_pred - y_true), max_error - K.abs(y_pred - y_true))), axis = -1)

def circular_mae(y_true, y_pred):
    max_error = tf.constant(360, dtype = 'float32')
    return K.mean(K.minimum(K.abs(y_pred - y_true), K.abs(max_error - K.abs(y_pred - y_true))), axis = -1)

def custom_mae(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true), axis = -1)

### Convert String into Reduction Metric Function

In [10]:
def get_Reduction_Metric(metric):
    
    if metric == 'custom_mae':
        return [custom_mae]
    elif metric == 'tf.keras.metrics.MeanAbsoluteError()':
        return [tf.keras.metrics.MeanAbsoluteError()]
    elif metric == 'circular_mae':
        return [circular_mae]
    elif metric == 'mean_squared_error':
        return ['mean_squared_error']
    else:
        assert(False, 'Metric yet unknown - Please modify get_Reduction_Metric to meet your requirements')
        return None

### Generierung Bottleneck-Features

In [11]:
def create_bottleneck_features(train_generator, valid_generator, bottleneck):
        
    print('Creating bottleneck features...')
    model = VGG16(include_top = False, weights = 'imagenet')
        
    #bottleneck_features_train = model.predict_generator(train_generator, train_generator.n // train_generator.batch_size)
    bottleneck_features_train = model.predict(train_generator)
    np.save(open(_LOG_DIR + 'Train_' + bottleneck, 'wb'), bottleneck_features_train)
        
    #bottleneck_features_valid = model.predict_generator(valid_generator, valid_generator.n // valid_generator.batch_size)
    bottleneck_features_valid = model.predict(valid_generator)
    np.save(open(_LOG_DIR + 'Valid_'  + bottleneck, 'wb'), bottleneck_features_valid)

### Generierung Datenpipeline (Angepasst für Talos)

In [12]:
def prepare_data(batch_size, num_samples, label_type):
    #print(_CSV_FILE)
    df = pd.read_csv(_CSV_FILE)
    df_shuffled = df.sample(frac = 1, random_state = 1)
    df_train = df_shuffled[0 : int(num_samples * 0.8 // batch_size * batch_size)]   
    df_valid = df_shuffled.drop(df_shuffled.index[0: df_train.shape[0]])[0 : int(num_samples * 0.2 // batch_size * batch_size)]
    
    train_labels = df_train.drop(['Filename RGB'], axis = 1).values
    valid_labels = df_valid.drop(['Filename RGB'], axis = 1).values
    
    bottleneck = 'Bottleneck_Features_{}_{}.npy'.format(str(num_samples), str(batch_size))                                       
    if not os.path.exists(_LOG_DIR + 'Train_' + bottleneck):
        
        if _USE_DATA_AUGMENTATION:
            train_data_generator = ImageDataGenerator(
                rescale = 1./255, 
                width_shift_range = 0.1,
                height_shift_range = 0.1, 
                zoom_range = 0.1,
                brightness_range = (0.25, 0.75),
                fill_mode = 'nearest'
            )
        else:
            train_data_generator = ImageDataGenerator(
                rescale = 1./255
            )
            
        valid_data_generator = ImageDataGenerator(
            rescale = 1./255
        )
    
        print('Y-Col: {}'.format(get_Label_Names(label_type)))
        
        train_generator = train_data_generator.flow_from_dataframe(
            dataframe = df_train,
            directory = _IMAGE_DIR,
            x_col = 'Filename RGB',
            y_col = get_Label_Names(label_type),
            class_mode = 'raw',
            target_size = (224, 224),
            color_mode = 'rgb',
            shuffle = False,
            seed = 1,
            batch_size = batch_size
        )
    
        valid_generator = valid_data_generator.flow_from_dataframe(
            dataframe = df_valid,
            directory = _IMAGE_DIR,
            x_col = 'Filename RGB',
            y_col = get_Label_Names(label_type),
            class_mode = 'raw',
            target_size = (224, 224),
            color_mode = 'rgb',
            shuffle = False,
            seed = 1,
            batch_size = batch_size
        )
                                                        
        create_bottleneck_features(train_generator, valid_generator, bottleneck)
        
    train_features = np.load(open(_LOG_DIR + 'Train_' + bottleneck, 'rb'))
    valid_features = np.load(open(_LOG_DIR + 'Valid_' + bottleneck, 'rb'))
                                                        
    return train_labels, valid_labels, train_features, valid_features

### Generierung Modell (Angepasst für Talos)

In [13]:
def grid_model_base(x, y, x_val, y_val, params):
    
    global _COUNTER    
    K.clear_session()

    train_labels, valid_labels, train_features, valid_features = prepare_data(params['batch_size'], params['samples'], params['label_type'])

    model = Sequential()
    model.add(Flatten(input_shape = train_features.shape[1 :])) # (7, 7, 512)
    
    dropout_rate = params['dropout']
    first_neuron = params['first_neuron']
    
    if params['activation'] == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = 0.1)
    elif params['activation'] == 'relu':
        activation_layer = ReLU()  
    
    model.add(Dense(units = first_neuron, kernel_initializer = glorot_uniform(seed = 1)))
    model.add(activation_layer)
    if dropout_rate > 0.0:
        model.add(Dropout(rate = dropout_rate))
        
    hidden_neuron_fraction = first_neuron
    for i in range(params['hidden_layers']):
        hidden_neuron_fraction = hidden_neuron_fraction // 2
        model.add(Dense(units = hidden_neuron_fraction, kernel_initializer = glorot_uniform(seed = 1)))
        model.add(activation_layer)
        if dropout_rate > 0.0:
            model.add(Dropout(rate = dropout_rate))
    
    model.add(Dense(units = 2, kernel_initializer = glorot_uniform(seed = 1)))

    print('Using Loss: {} \nand Reduction Metric: {}'.format(
        params['loss_function'], 
        get_Reduction_Metric(params['reduction_metric'])))
    
    model.compile(
        optimizer = params['optimizer'](lr = lr_normalizer(params['lr'], params['optimizer'])), 
        loss = params['loss_function'], 
        metrics = get_Reduction_Metric(params['reduction_metric'])
    )
    
    print('Monitor: {}'.format(params['monitor_value']))
    
    checkpointer = callbacks.ModelCheckpoint(
        filepath = _LOG_DIR + 'Best_Weights_FC_{}.hdf5'.format(_COUNTER),
        monitor = params['monitor_value'],
        verbose = 1,
        save_best_only = True,
        mode = 'min'
    )
    
    startTime = datetime.now()
    out = model.fit(
        x = train_features,
        y = train_labels,
        epochs = params['epochs'],
        validation_data = (valid_features, valid_labels),
        steps_per_epoch = int(params['samples'] * 0.8 // params['batch_size']),
        validation_steps = int(params['samples'] * 0.2 // params['batch_size']),
        callbacks = [checkpointer]
    )
    print("Time taken:", datetime.now() - startTime)
    
    _COUNTER = _COUNTER + 1
     
    return out, model

# Parameter <a name = "setup"></a><a href = #Top>Up</a></p>

### GridSerach

#### Hyper Parameter

In [14]:
#     Adam = RMSprop + Momentum (lr=0.001)
#     Nadam = Adam RMSprop + Nesterov-Momentum (lr=0.002)
#     RMSprop = (lr=0.001)
#     SGD = (lr=0.01)
#     Adagrad

hyper_parameter = {
    'samples': [20000],
    'epochs': [1],
    'batch_size': [32, 64],
    'optimizer': [Adam],
    'lr': [1, 2, 5],
    'first_neuron': [1024, 2048, 4096],
    'dropout': [0.25, 0.50],
    'activation': ['leakyrelu', 'relu'],
    'hidden_layers': [0, 1, 2, 3, 4],
    # beginning from here, Values should only contain a single entry:
    # ===============================================================
    'label_type': ['Angular'], # Stereographic, Angular, Normalized
    'loss_function': ['mean_squared_error'], # circular_mse
    'reduction_metric': ['custom_mae'], # tf.keras.metrics.MeanAbsoluteError(), circular_mae, custom_mae, mean_squared_error
    'monitor_value': ['val_custom_mae'] # val_custom_mae, val_mean_absolute_error, val_circular_mae, val_loss
}

In [15]:
def get_params():  
    return p

### Dateisystem

In [16]:
_RUN = 'SYNTH'
_LOSS = 'MSE'
_DATASET_NAME = '201019_2253_final'#'2020-05-28'
_DEVICE = 'GeForce_RTX_2080_Ti' #"/device:GPU:1" #'TITAN_GPU1'

storage = OutputDirectory.SSD # 'fast_output' if ssd storage may suffice, 'output' otherwise

#APPENDIX = 'Stereographic'

#FUNCTION_OVERRIDE = ['mean_squared_error', [custom_mae], 'val_custom_mae'] # None, or e. g. ['mean_squared_error', [circular_mae], 'val_circular_mae']

if hyper_parameter['label_type'][0] == 'Stereographic':
    _CSV_FILE_NAME = 'images_synthetisch_stereographic.csv'
    _STEREOGRAPHIC = True
elif hyper_parameter['label_type'][0] == 'Angular':
    _CSV_FILE_NAME = 'labels_ks_RGB.csv' #'images_synthetisch.csv'
    _STEREOGRAPHIC = False
elif hyper_parameter['label_type'][0] == 'Normalized':
    _CSV_FILE_NAME = 'images_synthetisch_normalized.csv'
    _STEREOGRAPHIC = False
else:
    assert(True, 'LabelType Invalid')

In [17]:
_USE_DATA_AUGMENTATION = False

In [18]:
_MODEL_NAME = '{}_Optimierung_Hyperparameter_v{}'.format(_DATASET_NAME, _RUN)
_IMAGE_DIR = '..\\..\\data_generation\\dataset\\{}\\'.format(_DATASET_NAME) # '..\\dataset\\{}\\'.format(_DATASET_NAME)
_CSV_FILE = _IMAGE_DIR + _CSV_FILE_NAME

_COUNTER = 0

_note = '_Custom-MAE'

_NET_DIR = '{}_Regression_{}\\{}_{}_Base{}\\'.format(_RUN, _LOSS, _DATASET_NAME, hyper_parameter['label_type'][0], _note)
_LOG_DIR = '..\\{}\\{}'.format(output_path[storage], _NET_DIR)

_RESULTS_FILE = '\\..\\{}_{}_Base{}_Results.csv'.format(_DATASET_NAME, hyper_parameter['label_type'][0], _note)

if(not os.path.exists(_LOG_DIR)):
    os.makedirs(_LOG_DIR)
else:
    input('Directory >>| {} |<< existiert bereits. Fortsetzen auf eigene Gefahr! (Weiter mit Enter)'.format(_LOG_DIR))

device_file = open(_LOG_DIR + '{}'.format(_DEVICE), "a+")
device_file.close()

Directory >>| ..\fast_output\SYNTH_Regression_MSE\201019_2253_final_Angular_Base_Custom-MAE\ |<< existiert bereits. Fortsetzen auf eigene Gefahr! (Weiter mit Enter)


# Ausführung GridSearch mit Talos <a name = "Base"></a><a href = #Top>Up</a></p>

In [16]:
import numpy as np
dummy_x = np.empty((1, 2, 3, 224, 224))
dummy_y = np.empty((1, 2))

with tf.device('/device:GPU:0'):
    
        t = ta.Scan(
            x = dummy_x,
            y = dummy_y,
            model = grid_model_base,
            params = hyper_parameter,
            experiment_name = '{}'.format(_DATASET_NAME),
            #shuffle=False,
            reduction_metric = hyper_parameter['reduction_metric'][0],
            disable_progress_bar = False,
            print_params = True,
            clear_session = True,
            save_weights = False
        )
        

t.data.to_csv(_LOG_DIR + _RESULTS_FILE, index = True)

  0%|                                                                                          | 0/360 [00:00<?, ?it/s]

{'activation': 'leakyrelu', 'batch_size': 32, 'dropout': 0.25, 'epochs': 1, 'first_neuron': 1024, 'hidden_layers': 0, 'label_type': 'Angular', 'loss_function': 'mean_squared_error', 'lr': 1, 'monitor_value': 'val_custom_mae', 'optimizer': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 'reduction_metric': 'custom_mae', 'samples': 20000}
Using Loss: mean_squared_error 
and Reduction Metric: [<function custom_mae at 0x0000024C33F2FE58>]
Monitor: val_custom_mae
Train on 16000 samples, validate on 4000 samples
15680/16000 [============================>.] - ETA: 0s - loss: 3821.8989 - custom_mae: 39.2101
Epoch 00001: val_custom_mae improved from inf to 31.90724, saving model to ..\fast_output\SYNTH_Regression_MSE\201019_2253_final_Angular_Base_Custom-MAE\Best_Weights_FC_0.hdf5
16000/16000 [==============================] - 5s 303us/sample - loss: 3799.5094 - custom_mae: 39.0489 - val_loss: 2938.1040 - val_custom_mae: 31.9072
Time taken: 0:00:06.534892


  0%|▏                                                                                 | 1/360 [00:08<52:15,  8.73s/it]

{'activation': 'leakyrelu', 'batch_size': 32, 'dropout': 0.25, 'epochs': 1, 'first_neuron': 1024, 'hidden_layers': 0, 'label_type': 'Angular', 'loss_function': 'mean_squared_error', 'lr': 2, 'monitor_value': 'val_custom_mae', 'optimizer': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 'reduction_metric': 'custom_mae', 'samples': 20000}
Using Loss: mean_squared_error 
and Reduction Metric: [<function custom_mae at 0x0000024C33F2FE58>]
Monitor: val_custom_mae
Train on 16000 samples, validate on 4000 samples
15936/16000 [============================>.] - ETA: 0s - loss: 3713.6342 - custom_mae: 38.4650
Epoch 00001: val_custom_mae improved from inf to 32.61966, saving model to ..\fast_output\SYNTH_Regression_MSE\201019_2253_final_Angular_Base_Custom-MAE\Best_Weights_FC_1.hdf5
16000/16000 [==============================] - 5s 285us/sample - loss: 3710.9920 - custom_mae: 38.4334 - val_loss: 3080.8014 - val_custom_mae: 32.6197
Time taken: 0:00:06.259690


  1%|▍                                                                                 | 2/360 [00:16<50:44,  8.50s/it]

...

 99%|█████████████████████████████████████████████████████████████████████████████▌| 358/360 [1:06:06<00:26, 13.49s/it]

{'activation': 'relu', 'batch_size': 64, 'dropout': 0.5, 'epochs': 1, 'first_neuron': 4096, 'hidden_layers': 4, 'label_type': 'Angular', 'loss_function': 'mean_squared_error', 'lr': 2, 'monitor_value': 'val_custom_mae', 'optimizer': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 'reduction_metric': 'custom_mae', 'samples': 20000}
Using Loss: mean_squared_error 
and Reduction Metric: [<function custom_mae at 0x0000024C33F2FE58>]
Monitor: val_custom_mae
Train on 16000 samples, validate on 3968 samples
15872/16000 [============================>.] - ETA: 0s - loss: 7309.6953 - custom_mae: 60.5210
Epoch 00001: val_custom_mae improved from inf to 48.78566, saving model to ..\fast_output\SYNTH_Regression_MSE\201019_2253_final_Angular_Base_Custom-MAE\Best_Weights_FC_358.hdf5
16000/16000 [==============================] - 10s 635us/sample - loss: 7289.3313 - custom_mae: 60.4253 - val_loss: 4449.2917 - val_custom_mae: 48.7857
Time taken: 0:00:11.863032


100%|█████████████████████████████████████████████████████████████████████████████▊| 359/360 [1:06:20<00:13, 13.56s/it]

{'activation': 'relu', 'batch_size': 64, 'dropout': 0.5, 'epochs': 1, 'first_neuron': 4096, 'hidden_layers': 4, 'label_type': 'Angular', 'loss_function': 'mean_squared_error', 'lr': 5, 'monitor_value': 'val_custom_mae', 'optimizer': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 'reduction_metric': 'custom_mae', 'samples': 20000}
Using Loss: mean_squared_error 
and Reduction Metric: [<function custom_mae at 0x0000024C33F2FE58>]
Monitor: val_custom_mae
Train on 16000 samples, validate on 3968 samples
15936/16000 [============================>.] - ETA: 0s - loss: 25274.0337 - custom_mae: 71.0742
Epoch 00001: val_custom_mae improved from inf to 53.35452, saving model to ..\fast_output\SYNTH_Regression_MSE\201019_2253_final_Angular_Base_Custom-MAE\Best_Weights_FC_359.hdf5
16000/16000 [==============================] - 10s 631us/sample - loss: 25197.6372 - custom_mae: 71.0079 - val_loss: 5392.3791 - val_custom_mae: 53.3545
Time taken: 0:00:11.798586


100%|██████████████████████████████████████████████████████████████████████████████| 360/360 [1:06:34<00:00, 11.09s/it]


# Copy best 10 Results to NAS if SSD Directory was selected

In [19]:
best_results = _LOG_DIR + '..\\{}_{}_Base{}_Results.csv'.format(_DATASET_NAME, hyper_parameter['label_type'][0], _note)
df = pd.read_csv(best_results).drop(columns = ['round_epochs', 'samples', 'epochs'], axis = 0)
df = df.sort_values(hyper_parameter['monitor_value'][0], axis = 0, ascending = True, inplace = False, kind = 'quicksort', na_position = 'last')
print('Displaying: {}'.format(best_results))
df.head(10)

Displaying: ..\fast_output\SYNTH_Regression_MSE\201019_2253_final_Angular_Base_Custom-MAE\..\201019_2253_final_Angular_Base_Custom-MAE_Results.csv


,Unnamed: 0,start,end,duration,loss,custom_mae,val_loss,val_custom_mae,activation,batch_size,dropout,first_neuron,hidden_layers,label_type,loss_function,lr,monitor_value,optimizer,reduction_metric
32,32,11/04/20-133756,11/04/20-133811,15.101719,4383.826411,40.741211,2547.475186,28.446991,leakyrelu,32,0.25,4096,0,Angular,mean_squared_error,5,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
123,123,11/04/20-135632,11/04/20-135645,12.903348,4370.115269,41.885189,2500.798960,28.509195,leakyrelu,64,0.25,4096,1,Angular,mean_squared_error,1,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
17,17,11/04/20-133503,11/04/20-133513,10.042880,4001.966997,39.212330,2567.742597,28.672417,leakyrelu,32,0.25,2048,0,Angular,mean_squared_error,5,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
37,37,11/04/20-133916,11/04/20-133932,16.196169,5342.255862,44.598431,2508.385596,29.009453,leakyrelu,32,0.25,4096,2,Angular,mean_squared_error,2,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
126,126,11/04/20-135712,11/04/20-135725,13.103501,4892.983958,44.019562,2545.006084,29.102833,leakyrelu,64,0.25,4096,2,Angular,mean_squared_error,1,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
109,109,11/04/20-135414,11/04/20-135423,8.591999,4502.218566,42.888409,2549.238323,29.138407,leakyrelu,64,0.25,2048,1,Angular,mean_squared_error,2,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
24,24,11/04/20-133619,11/04/20-133629,10.735754,4304.691335,43.828423,2479.888127,29.194164,leakyrelu,32,0.25,2048,3,Angular,mean_squared_error,1,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
36,36,11/04/20-133900,11/04/20-133916,16.164278,4115.395587,41.639866,2575.603947,29.298376,leakyrelu,32,0.25,4096,2,Angular,mean_squared_error,1,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
108,108,11/04/20-135405,11/04/20-135414,8.623027,3881.180195,40.146988,2558.554630,29.299927,leakyrelu,64,0.25,2048,1,Angular,mean_squared_error,1,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae
18,18,11/04/20-133514,11/04/20-133524,10.283503,3871.796719,39.797153,2617.595140,29.322357,leakyrelu,32,0.25,2048,1,Angular,mean_squared_error,1,val_custom_mae,<class 'tensorflow.python.keras.optimizer_v2.a...,custom_mae


In [20]:
def copy_base_directory(src, dst, data, symlinks = False, ignore = None):
    maxLen = 0
    message = ''
    
    networks_to_copy = data.head(10).index
    
    if not os.path.exists(dst):
        
        message = 'Creating Path: {}'.format(src)
        maxLen = max(maxLen, len(message))
        print(message + ' ' * (maxLen - len(message)), end = '\r')
        
        os.makedirs(dst)
        
    for item in os.listdir(src):
        
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        
        if os.path.isdir(s):
            
            message = 'Copying Directory: {}'.format(s)
            maxLen = max(maxLen, len(message))
            print(message + ' ' * (maxLen - len(message)), end = '\r')
            
            shutil.copytree(s, d, symlinks, ignore)
            
        else:
            
            if not os.path.exists(d): #or os.stat(s).st_mtime - os.stat(d).st_mtime > 1:
                for idx in networks_to_copy:
                    net = '_FC_{}.hdf5'.format(idx)
                    if net in item or '_Bottleneck_' in item or '_GPU' in item:
                        message = 'Copying File: {}'.format(s)
                        maxLen = max(maxLen, len(message))
                        print(message + ' ' * (maxLen - len(message)), end = '\r')
                
                        shutil.copy2(s, d)
        
        time.sleep(.1)    
    
    src_csv = src + _RESULTS_FILE
    dst_csv = dst + _RESULTS_FILE
    if not os.path.exists(dst_csv):
        message = 'Copying File: {}'.format(src_csv)
        maxLen = max(maxLen, len(message))
        print(message + ' ' * (maxLen - len(message)), end = '\r')
        shutil.copy2(src_csv, dst_csv)
     
    message = 'Coyping... Done'
    maxLen = max(maxLen, len(message))
    print(message + ' ' * (maxLen - len(message)), end = '\n')

    
def delete_directory(src, terminator = '\n'):
    message = ''
    maxLen = 0
    
    try:
        message = 'Deleting {}'.format(src)
        maxLen = max(maxLen, len(message))
        print(message + ' ' * (maxLen - len(message)), end = '\r')
        
        if os.path.isdir(src):
            shutil.rmtree(src)
            
        elif os.path.isfile(src):
            os.remove(src)
        
    except OSError as e:
        message = 'Error: {} : {}'.format(src, e.strerror)
        maxLen = max(maxLen, len(message))
        print(message + ' ' * (maxLen - len(message)), end = '\n')
        return
    
    message = 'Deleting... Done'
    maxLen = max(maxLen, len(message))
    print(message + ' ' * (maxLen - len(message)), end = terminator)

    
def copy_base_training(src, dst):
    copy_base_directory(src, dst, df)
    
    delete_directory(src, terminator = '\r')
    delete_directory(src + _RESULTS_FILE, terminator = '\r')
    if not os.listdir(src + '..\\'):
        delete_directory(src + '..\\', terminator = '\r')

In [21]:
if(storage == OutputDirectory.SSD):
    _COPY_DIR = '..\\output\\{}'.format(_NET_DIR)
    copy_base_training(_LOG_DIR, _COPY_DIR)

Coyping... Done                                                                                                                                      
